In [1]:
# Second data compilation file, reads in the initialSet csv file populated with PDB_IDs & UniProt_IDs and queries/calculates the following features:

# "Surface_area", "sequence_length", "num_of_helicies", "num_of_strand",
#    "total_helix_length", "total_Strand_length", "percent_helix", "percent_strand",
#    "longest_helix_length", "longest_strand_length", "Molecular_weight", "Hydrophobicity",
#    "ec_numbers", "sequence",
#    "Hydrophobic_AA_percent", "Polar_AA_percent", "Basic_AA_percent",
#    "Acidic_AA_percent", "Special_AA_percent", "Num_of_Chains", "SA_div_total_seq"

# It then appends them to the initialSet csv file - can pick up from where previous run left off (may take a few minutes to start back up)

In [2]:
import requests
import pandas as pd
import json
import os
import numpy as np
from Bio.SeqUtils.ProtParam import ProteinAnalysis
from Bio.PDB import PDBParser, PDBList
from Bio.PDB.SASA import ShrakeRupley
from xml.etree import ElementTree as ET

# Define file paths
input_file = "/Users/carmenshero/Desktop/Datasets/initialSet.csv"

# Load dataset correctly (remove duplicate `pd.read_csv`)
df_training = pd.read_csv(input_file, dtype={"ec_numbers": str, "sequence": str}, low_memory=False)

# Define the exact column order for features (C to W)
feature_columns = [
    "Surface_area", "sequence_length", "num_of_helicies", "num_of_strand",
    "total_helix_length", "total_Strand_length", "percent_helix", "percent_strand",
    "longest_helix_length", "longest_strand_length", "Molecular_weight", "Hydrophobicity",
    "ec_numbers", "sequence",
    "Hydrophobic_AA_percent", "Polar_AA_percent", "Basic_AA_percent",
    "Acidic_AA_percent", "Special_AA_percent", "Num_of_Chains", "SA_div_total_seq"
]

# Add missing columns with NaN
for col in feature_columns:
    if col not in df_training.columns:
        df_training[col] = np.nan

# Reorder columns to make sure PDB_ID, UniProt_ID come first, followed by feature columns
column_order = ["PDB_ID", "UniProt_ID"] + feature_columns
df_training = df_training[[col for col in column_order if col in df_training.columns]]

# Replace "nan" and NaN with an empty string (ensures missing values are recognized)
df_training["ec_numbers"] = df_training["ec_numbers"].replace(["nan", np.nan], "")
df_training["sequence"] = df_training["sequence"].replace(["nan", np.nan], "")

# Fix "Processed" logic to check only C-P but always reprocess empty sequences
df_training["Processed"] = df_training.iloc[:, 2:23].notnull().all(axis=1) & df_training["sequence"].str.strip().astype(bool)

# Identify rows that still need processing
remaining_rows = df_training[~df_training["Processed"]].copy()
df_training.drop(columns=["Processed"], inplace=True)

In [3]:
# Function to fetch UniProt info
def get_protein_info(uniprot_id):
    url = f"https://rest.uniprot.org/uniprotkb/{uniprot_id}.json"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    return None

# Function to get protein sequence
def get_protein_sequence(uniprot_id):
    data = get_protein_info(uniprot_id)
    if data:
        return data.get('sequence', {}).get('value', 'N/A')
    return 'N/A'

# Function to get EC numbers
def extract_ec_numbers(protein_info):
    ec_numbers = []
    if protein_info:
        for comment in protein_info.get('comments', []):
            if comment.get('commentType') == 'CATALYTIC ACTIVITY':
                reaction = comment.get('reaction', {})
                ec_number = reaction.get('ecNumber')
                if ec_number:
                    ec_numbers.append(ec_number)
    return ";".join(ec_numbers) if ec_numbers else "MISSING"

# Function to get secondary structure info
def get_pdbml(pdb_id):
    url = f"https://files.rcsb.org/view/{pdb_id}.xml"
    response = requests.get(url)
    return response.content if response.status_code == 200 else None

def parse_secondary_structure(pdbml_content):
    if not pdbml_content:
        return []
    
    root = ET.fromstring(pdbml_content)
    ns = {'pdbx': 'http://pdbml.pdb.org/schema/pdbx-v50.xsd'}
    sec_struct = []

    for ss in root.findall('.//pdbx:struct_conf', ns):
        start_res = ss.find('pdbx:beg_auth_seq_id', ns).text
        end_res = ss.find('pdbx:end_auth_seq_id', ns).text
        conf_type = ss.find('pdbx:conf_type_id', ns).text
        sec_struct.append((int(start_res), int(end_res), conf_type))
    
    for ss in root.findall('.//pdbx:struct_sheet_range', ns):
        start_res = ss.find('pdbx:beg_auth_seq_id', ns).text
        end_res = ss.find('pdbx:end_auth_seq_id', ns).text
        sec_struct.append((int(start_res), int(end_res), 'STRAND'))
    
    return sec_struct

# Calculate surface area
def Get_SA(pdb_id):
    url = f"https://files.rcsb.org/download/{pdb_id}.pdb"
    file_path = f"/Users/carmenshero/Desktop/{pdb_id}.pdb"

    response = requests.get(url)
    if response.status_code != 200:
        return np.nan

    with open(file_path, 'wb') as file:
        file.write(response.content)

    parser = PDBParser(QUIET=True)
    structure = parser.get_structure(pdb_id, file_path)
    sr = ShrakeRupley()
    sr.compute(structure, level="S")
    surface_area = structure.sasa

    os.remove(file_path)  # Cleanup
    return int(surface_area)

# Define Amino Acid Groups
aa_groups = {
    'Hydrophobic': ['A', 'V', 'I', 'L', 'M', 'F', 'W', 'Y'],
    'Polar': ['S', 'T', 'N', 'Q', 'C'],
    'Basic': ['K', 'R', 'H'],
    'Acidic': ['D', 'E'],
    'Special': ['G', 'P']
   # 'Other': ['X', 'U', 'B', 'Z']
}

# Function to Calculate Amino Acid Composition
from itertools import chain

def amino_acid_composition(sequence):
    if sequence == "N/A" or sequence == "":
        return {aa: 0.0 for aa in chain.from_iterable(aa_groups.values())}

    # Include all amino acids from all groups, including 'Other'
    amino_acid_counts = {aa: 0 for aa in chain.from_iterable(aa_groups.values())}

    for amino_acid in sequence:
        if amino_acid in amino_acid_counts:
            amino_acid_counts[amino_acid] += 1
        else:
            # Optional: warn about truly unknown characters
            print(f"Unknown amino acid '{amino_acid}' skipped.")

    sequence_length = len(sequence)
    if sequence_length == 0:
        return {aa: 0.0 for aa in amino_acid_counts}

    return {aa: (count / sequence_length) * 100 for aa, count in amino_acid_counts.items()}

# Function to Group Amino Acid Frequencies into Categories
def group_aa_frequencies(aa_percent):
    grouped_data = {group: 0 for group in aa_groups.keys()}
    for aa, freq in aa_percent.items():
        for group, aas in aa_groups.items():
            if aa in aas:
                grouped_data[group] += freq
                break
    return grouped_data

# Calculate molecular weight and hydrophobicity
from Bio.SeqUtils.ProtParam import ProteinAnalysis
import re

# Calculate molecular weight and hydrophobicity
def calculate_physicochemical_properties(sequence):
    if sequence == "N/A":
        return np.nan, np.nan
    
    analysed_seq = ProteinAnalysis(sequence)
    return analysed_seq.molecular_weight(), analysed_seq.gravy()

# Calculate number of chains (also needed for SA_div_total_seq)
def get_num_of_chains(pdb_id):
    """Retrieve the number of chains from a PDB file."""
    pdbl = PDBList()
    pdb_file = pdbl.retrieve_pdb_file(pdb_id, file_format='pdb')
    
    # Parse the PDB file
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure(pdb_id, pdb_file)
    
    # Count unique chain IDs
    num_chains = len(set(chain.id for model in structure for chain in model))
    
    # Clean up: Delete PDB file after use
    os.remove(pdb_file)

    return num_chains

In [4]:
# Process each row
for index, row in remaining_rows.iterrows():
    pdb_id, uniprot_id = row["PDB_ID"], row["UniProt_ID"]

    # Skip rows with missing UniProt ID
    if not uniprot_id or pd.isna(uniprot_id):
        print(f"Skipping {pdb_id} due to missing UniProt ID.")
        continue

    try:
        # Fetch data
        protein_info = get_protein_info(uniprot_id)
        sequence = get_protein_sequence(uniprot_id)
        secondary_structure = parse_secondary_structure(get_pdbml(pdb_id))
        surface_area = Get_SA(pdb_id)
        num_of_chains = get_num_of_chains(pdb_id)

        # Calculate Amino Acid Composition
        aa_percent = amino_acid_composition(sequence)
        grouped_frequencies = group_aa_frequencies(aa_percent)

        # Calculate structural properties
        total_helix_length = sum(end - start + 1 for start, end, conf in secondary_structure if "HELX" in conf)
        total_strand_length = sum(end - start + 1 for start, end, conf in secondary_structure if conf == "STRAND")
        sequence_length = len(sequence) if sequence != "N/A" else np.nan
        percent_helix = round((total_helix_length / sequence_length) * 100, 2) if sequence_length else np.nan
        percent_strand = round((total_strand_length / sequence_length) * 100, 2) if sequence_length else np.nan
        longest_helix_length = max((end - start + 1 for start, end, conf in secondary_structure if "HELX" in conf), default=0)
        longest_strand_length = max((end - start + 1 for start, end, conf in secondary_structure if conf == "STRAND"), default=0)
        molecular_weight, hydrophobicity = calculate_physicochemical_properties(sequence)

        # Calculate SA_div_total_seq
        if num_of_chains > 0 and sequence_length > 0:  # Avoid division by zero
            SA_div_total_seq = surface_area / (sequence_length / num_of_chains)
        else:
            SA_div_total_seq = np.nan  # Assign NaN if division is invalid

        # Store updated values in the dataframe
        df_training.at[index, "Surface_area"] = surface_area
        df_training.at[index, "sequence_length"] = sequence_length
        df_training.at[index, "num_of_helicies"] = sum(1 for _, _, conf in secondary_structure if "HELX" in conf)
        df_training.at[index, "num_of_strand"] = sum(1 for _, _, conf in secondary_structure if conf == "STRAND")
        df_training.at[index, "total_helix_length"] = total_helix_length
        df_training.at[index, "total_Strand_length"] = total_strand_length
        df_training.at[index, "percent_helix"] = percent_helix
        df_training.at[index, "percent_strand"] = percent_strand
        df_training.at[index, "longest_helix_length"] = longest_helix_length
        df_training.at[index, "longest_strand_length"] = longest_strand_length
        df_training.at[index, "Molecular_weight"] = molecular_weight
        df_training.at[index, "Hydrophobicity"] = hydrophobicity
        df_training.at[index, "ec_numbers"] = extract_ec_numbers(protein_info)  # Always returns a string
        df_training.at[index, "sequence"] = sequence  # Store protein sequence
        
        # Assign Amino Acid Percentages (Columns Q-U)
        df_training.at[index, "Hydrophobic_AA_percent"] = grouped_frequencies["Hydrophobic"]
        df_training.at[index, "Polar_AA_percent"] = grouped_frequencies["Polar"]
        df_training.at[index, "Basic_AA_percent"] = grouped_frequencies["Basic"]
        df_training.at[index, "Acidic_AA_percent"] = grouped_frequencies["Acidic"]
        df_training.at[index, "Special_AA_percent"] = grouped_frequencies["Special"]
        df_training.at[index, "Num_of_Chains"] = num_of_chains
        df_training.at[index, "SA_div_total_seq"] = SA_div_total_seq
        
        # Save progress after processing each row
        df_training.to_csv(input_file, index=False)  
        print(f"✔ Updated {pdb_id} ({uniprot_id}) and saved progress.")

    except Exception as e:
        print(f"Error processing {pdb_id} ({uniprot_id}): {e}")
        continue  # Skip this row and move to the next


✔ Updated 1A1T (Q75677) and saved progress.
✔ Updated 1A40 (P0AG82) and saved progress.
✔ Updated 1A45 (P23005) and saved progress.
✔ Updated 1A5P (P61823) and saved progress.
✔ Updated 1A73 (Q94702) and saved progress.
✔ Updated 1A77 (Q58839) and saved progress.
✔ Updated 1A7B (P08921) and saved progress.
✔ Updated 1A87 (P08083) and saved progress.
✔ Updated 1A8Q (P33912) and saved progress.
✔ Updated 1A9T (P55859) and saved progress.
✔ Updated 1A9V (P49278) and saved progress.
✔ Updated 1ACD (P04117) and saved progress.
✔ Updated 1ACW (P56215) and saved progress.
✔ Updated 1AD7 (P07231) and saved progress.
